# **Local Interpretable Model-Agnostic Explanations**
We use the LIME package to visualize the attention of our model in the form of concept slices.

Install LIME package

In [ ]:
!pip install lime

Import necessary packages

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os
from scipy import ndimage

Add your image path in the cell below

In [ ]:
img_path = 'YOUR IMAGE PATH'

Preprocess input

In [ ]:
x = plt.imread(img_path)
plt.imshow(x)
x = x/255
x = np.expand_dims(x,0)
x.shape

Load your Keras model below

In [ ]:
# Add model path below
model = load_model('YOUR MODEL PATH')

Get model predictions

In [ ]:
model.predict(x)

Get explanations

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(x[0], model.predict)
temp, mask = explanation.get_image_and_mask(0, positive_only=True, hide_rest=True)

Visualize results

In [ ]:
plt.imshow(mark_boundaries(temp, mask))
plt.axis('off')

# **Testing Sensitivity to Gaussian Noise and Affine Transformations**

Function outlining the perturbed input

In [ ]:
def noise(img,value):
    if(value==0):
        t = img
        t = t[:,:,::-1]
    if(value==1):
        t = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
        t = t[:,:,::-1]
    if(value==2):
        t = ndimage.rotate(img,90,reshape=False)
        t = t[:,:,::-1]
    if(value==3):
        t = ndimage.rotate(img,180,reshape=False)
        t = t[:,:,::-1]
    if(value==4):
        t = ndimage.rotate(img,270,reshape=False)
        t = t[:,:,::-1]
    return t

Function operating LIME on perturbed input

In [ ]:
def lime_noise(img,value):
    x = noise(img,value)
    x = (x/255).astype(np.float32)
    x = np.expand_dims(x, axis=0)
    explanation = explainer.explain_instance(x[0], model.predict)
    temp, mask = explanation.get_image_and_mask(0, positive_only=True, hide_rest=True)
    gg = mark_boundaries(temp, mask)
    return gg

Visualize results

In [ ]:
img = cv2.imread(img_path)
explainer = lime_image.LimeImageExplainer()
fig = plt.figure(figsize=(50, 50), facecolor='white')

for i in range(1,11):
    plt.subplot(2,5,i)
    if i<6:
        plt.imshow(noise(img, i-1))
    else:
        plt.imshow(lime_noise(img,i-6))
    plt.axis('off')
    fig.tight_layout(h_pad=0, w_pad=0)
    plt.subplots_adjust(top = 0.99, bottom=0.01, wspace=0, hspace=0)